In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder, RobustScaler

In [3]:
pwd

'C:\\Users\\635718\\Desktop\\DataScience\\HackerEarth\\FutureDatathon'

In [10]:
sample = pd.read_csv('./Q2RecData/sampleSubmission.csv')

In [ ]:
sample.head(2)

In [5]:
requiredCustomer = sample['customerID']

In [6]:
requiredCustomer.shape

(39205,)

In [17]:
# load product and sample file
products = pd.read_csv('./Q2RecData/products.csv', nrows=500)

In [18]:
products.shape

(500, 16)

In [19]:
products.nunique()

customerID                     25
DOB                            25
Gender                          2
State                           2
PinCode                        17
transactionDate                58
store_code                      4
store_description               4
till_no                        17
transaction_number_by_till     69
promo_code                     77
promotion_description          76
product_code                  376
product_description           376
sale_price_after_promo        279
discountUsed                    1
dtype: int64

In [20]:
products.head(2)

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,transaction_number_by_till,promo_code,promotion_description,product_code,product_description,sale_price_after_promo,discountUsed
0,BBID_2041,1981-10-18,male,NaN,452001,2016-01-19,2655,BB-INDORE-MALHAR MEGA MALL,2,51535,NONPROMO,NaN,300785147,BB-CB-16X20X168SWG-Suitable for ROI New,2.0,Payback
1,BBID_2041,1981-10-18,male,NaN,452001,2016-06-21,2655,BB-INDORE-MALHAR MEGA MALL,21,57836,NONPROMO,NaN,1000315939,SOO FRESH ROLL PIZZA 1PLT,40.0,Payback


In [60]:
#products[(products['customerID'] == 'BBID_2049382') | (products['customerID'] == 'BBID_2049678')]
products[(products['transaction_number_by_till'] == 632)]

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,transaction_number_by_till,promo_code,promotion_description,product_code,product_description,sale_price_after_promo,discountUsed
283,BBID_20415,1986-07-19,male,MADHYA PRADESH,462016,2016-04-28,2906,BB-INDORE-TREASURE ISLAND,11,632,NONPROMO,NaN,1000328475001,"SCF-3151-LG112-PR-RY-SE, 50x180, NAVY",199.0,Payback
284,BBID_20415,1986-07-19,male,MADHYA PRADESH,462016,2016-04-28,2906,BB-INDORE-TREASURE ISLAND,11,632,NONPROMO,NaN,1000254413006,"DJWV-4161-RU-554-SO-LC, S, TAUPE",799.0,Payback
291,BBID_20415,1986-07-19,male,MADHYA PRADESH,462016,2016-04-28,2906,BB-INDORE-TREASURE ISLAND,11,632,NONPROMO,NaN,1000531410,SOFRSH VD COOKIES ALMOND PST TB 200G,80.0,Payback
294,BBID_20415,1986-07-19,male,MADHYA PRADESH,462016,2016-04-28,2906,BB-INDORE-TREASURE ISLAND,11,632,NONPROMO,NaN,1000531236,SOFRSH VD COOKIES PISTA TB 200G,70.0,Payback
319,BBID_20415,1986-07-19,male,MADHYA PRADESH,462016,2016-04-28,2906,BB-INDORE-TREASURE ISLAND,11,632,NONPROMO,NaN,1000435159002,"BFTS-4161-BF03-C-HS-RF-, M, SEA BLUE MEL",599.0,Payback
368,BBID_20415,1986-07-19,male,MADHYA PRADESH,462016,2016-04-28,2906,BB-INDORE-TREASURE ISLAND,11,632,NONPROMO,NaN,1000460036006,"SPTP-0010-1673-A-FL-SF, S, NAVY HEATHER",699.0,Payback


In [132]:
products.groupby(by='customerID')['product_code'].count().sort_values(ascending=False).head(2)
#count().sort_values(ascending=False).head() #BBID_20445646    

customerID
BBID_20415    137
BBID_20413     67
Name: product_code, dtype: int64

In [23]:
misfit_customers = list(set(sample['customerID']) - set(products['customerID']))

In [24]:
len(misfit_customers)

39203

In [25]:
payment = pd.read_csv('./Q2RecData/tenderModes.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [57]:
payment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866277 entries, 0 to 866276
Data columns (total 13 columns):
customerID                    866277 non-null object
DOB                           837289 non-null object
Gender                        774742 non-null object
State                         755746 non-null object
PinCode                       797283 non-null object
transactionDate               866277 non-null object
store_code                    866277 non-null int64
store_description             866277 non-null object
till_no                       866277 non-null int64
tender_type                   866277 non-null object
transaction_number_by_till    866277 non-null int64
payment_amount_by_tender      866277 non-null float64
PaymentUsed                   859212 non-null object
dtypes: float64(1), int64(3), object(9)
memory usage: 56.2+ MB


In [58]:
payment.head(2)

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,tender_type,transaction_number_by_till,payment_amount_by_tender,PaymentUsed
0,BBID_2041,1981-10-18,male,NaN,452001,2016-01-19,2655,BB-INDORE-MALHAR MEGA MALL,2,EDCHDFC,51535,672.0,HDFC CREDIT CARD
1,BBID_2041,1981-10-18,male,NaN,452001,2016-04-07,2655,BB-INDORE-MALHAR MEGA MALL,21,PYOC,43394,35.0,Payback


In [20]:
#Change the data to form clusters as we did in first part
#Then we predict the values for each cluster seperately

In [27]:
#Missing Data
total = products.isnull().sum().sort_values(ascending=False)
percent = (products.isnull().sum()/products.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(6)

,Total,Percent
promotion_description,390,0.78
State,225,0.45
Gender,75,0.15
discountUsed,0,0.00
sale_price_after_promo,0,0.00
product_description,0,0.00


In [28]:
drop_columns_products = ['State','PinCode','store_description','till_no','promo_code','promotion_description',
                         'product_description','discountUsed','sale_price_after_promo']
products_cleaned = products.drop(drop_columns_products, axis=1)

In [29]:
#Payment Missing Data
total = payment.isnull().sum().sort_values(ascending=False)
percent = (payment.isnull().sum()/payment.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(6)

,Total,Percent
State,110531,0.127593
Gender,91535,0.105665
PinCode,68994,0.079644
DOB,28988,0.033463
PaymentUsed,7065,0.008156
payment_amount_by_tender,0,0.000000


In [30]:
drop_columns_payment = ['DOB','Gender','State','PinCode','store_description','till_no','tender_type','PaymentUsed']
payment_cleaned = payment.drop(drop_columns_payment, axis=1)

In [31]:
payment_cleaned.columns

Index(['customerID', 'transactionDate', 'store_code',
       'transaction_number_by_till', 'payment_amount_by_tender'],
      dtype='object')

In [32]:
products_cleaned.head(2)

,customerID,DOB,Gender,transactionDate,store_code,transaction_number_by_till,product_code
0,BBID_2041,1981-10-18,male,2016-01-19,2655,51535,300785147
1,BBID_2041,1981-10-18,male,2016-06-21,2655,57836,1000315939


In [33]:
payment_cleaned.head(2)

,customerID,transactionDate,store_code,transaction_number_by_till,payment_amount_by_tender
0,BBID_2041,2016-01-19,2655,51535,672.0
1,BBID_2041,2016-04-07,2655,43394,35.0


In [34]:
#grouping the data in the payment to combine the amount per transaction
payment_cleaned['totalSpend'] = payment_cleaned.groupby(['customerID','transactionDate','store_code',
                                                         'transaction_number_by_till']).transform('sum')

In [70]:
payment_cleaned.head(2)

,customerID,transactionDate,store_code,transaction_number_by_till,totalSpend
0,BBID_2041,2016-01-19,2655,51535,672.0
1,BBID_2041,2016-04-07,2655,43394,35.0


In [133]:
payment[payment['payment_amount_by_tender'] < 0].head(2)

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,tender_type,transaction_number_by_till,payment_amount_by_tender,PaymentUsed
13,BBID_2043,1978-05-03,male,NaN,625016,2016-06-08,4986,BB-MADURAI-KOCHADAI VILLAGE,4,CR-NOTE(I),23523,-215.0,CREDIT NOTE (ISSUES)
42,BBID_2046,1959-11-01,male,NaN,452009,2016-05-02,2906,BB-INDORE-TREASURE ISLAND,21,CR-NOTE(I),978,-808.0,CREDIT NOTE (ISSUES)


In [36]:
payment_cleaned.drop('payment_amount_by_tender', axis=1, inplace=True)

In [134]:
payment_cleaned.drop_duplicates(keep='first', inplace=True)

In [135]:
payment.shape

(866277, 13)

In [136]:
payment_cleaned.shape

(683778, 5)

In [41]:
products_cleaned.shape

(500, 7)

In [42]:
#Merge the product and payment data
merge_on = ['customerID','transactionDate','store_code','transaction_number_by_till']
merged_data = pd.merge(products_cleaned,payment_cleaned,how='left',on=merge_on)

In [43]:
merged_data.shape

(500, 8)

In [44]:
merged_data.head(2)

,customerID,DOB,Gender,transactionDate,store_code,transaction_number_by_till,product_code,totalSpend
0,BBID_2041,1981-10-18,male,2016-01-19,2655,51535,300785147,672.0
1,BBID_2041,1981-10-18,male,2016-06-21,2655,57836,1000315939,40.0


In [45]:
merged_data['customerID'].value_counts().head(2)

BBID_20415    137
BBID_20413     67
Name: customerID, dtype: int64

In [46]:
#add missing data
merged_data['totalSpend'].fillna(merged_data['totalSpend'].mean(), inplace=True)
merged_data['Gender'].fillna('male', inplace=True)
merged_data['DOB'].fillna(pd.to_datetime("1991-12-03"), inplace=True)

In [47]:
#Change Gender to categorical
merged_data['Gender'] = merged_data['Gender'].map( {'female': 0, 'male': 1} ).astype(int)

In [48]:
merged_data.loc[merged_data['DOB'] == 'NANA','DOB'] = pd.to_datetime("1991-12-03")

In [49]:
#get Age from DOB
merged_data['age'] = pd.to_datetime('today').year - pd.to_datetime(merged_data['DOB']).dt.year

In [50]:
#get day of purchase from transactionDate
merged_data['day_of_transaction'] = pd.to_datetime(merged_data['transactionDate']).dt.weekday

In [52]:
merged_data.head(2)

,customerID,DOB,Gender,transactionDate,store_code,transaction_number_by_till,product_code,totalSpend,age,day_of_transaction
0,BBID_2041,1981-10-18,1,2016-01-19,2655,51535,300785147,672.0,36,1
1,BBID_2041,1981-10-18,1,2016-06-21,2655,57836,1000315939,40.0,36,1


In [54]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 10 columns):
customerID                    500 non-null object
DOB                           500 non-null object
Gender                        500 non-null int32
transactionDate               500 non-null object
store_code                    500 non-null int64
transaction_number_by_till    500 non-null int64
product_code                  500 non-null int64
totalSpend                    500 non-null float64
age                           500 non-null int64
day_of_transaction            500 non-null int64
dtypes: float64(1), int32(1), int64(5), object(3)
memory usage: 35.2+ KB


In [56]:
merged_data['transactionDate'] = pd.to_datetime(merged_data['transactionDate'])

In [63]:
merged_data[merged_data['transaction_number_by_till'] == 632]

,customerID,DOB,Gender,transactionDate,store_code,transaction_number_by_till,product_code,totalSpend,age,day_of_transaction
283,BBID_20415,1986-07-19,1,2016-04-28,2906,632,1000328475001,2446.0,31,3
284,BBID_20415,1986-07-19,1,2016-04-28,2906,632,1000254413006,2446.0,31,3
291,BBID_20415,1986-07-19,1,2016-04-28,2906,632,1000531410,2446.0,31,3
294,BBID_20415,1986-07-19,1,2016-04-28,2906,632,1000531236,2446.0,31,3
319,BBID_20415,1986-07-19,1,2016-04-28,2906,632,1000435159002,2446.0,31,3
368,BBID_20415,1986-07-19,1,2016-04-28,2906,632,1000460036006,2446.0,31,3


In [67]:
merged_data[merged_data['customerID'] == 'BBID_20415'].sort_values(by='transactionDate').head(2)

,customerID,DOB,Gender,transactionDate,store_code,transaction_number_by_till,product_code,totalSpend,age,day_of_transaction
331,BBID_20415,1986-07-19,1,2015-08-11,2655,1351,108100362,1949.58,31,1
281,BBID_20415,1986-07-19,1,2015-08-11,2655,1351,1000145745011,1949.58,31,1


In [69]:
merged_data[merged_data['totalSpend'] < 0]

,customerID,DOB,Gender,transactionDate,store_code,transaction_number_by_till,product_code,totalSpend,age,day_of_transaction
59,BBID_2043,1978-05-03,1,2016-06-08,4986,23523,108037613,-215.0,39,2
129,BBID_2046,1959-11-01,1,2016-05-02,2906,978,1000455039005,-808.0,58,0
146,BBID_2046,1959-11-01,1,2016-05-02,2906,978,1000058082,-808.0,58,0
287,BBID_20415,1986-07-19,1,2016-11-17,2655,65993,1000477316012,-599.0,31,3
308,BBID_20415,1986-07-19,1,2015-08-13,2655,1433,1000150037001,-599.0,31,3


In [74]:
#Drop the final set of columns
merged_data.drop(['DOB','transaction_number_by_till'], axis=1, inplace=True)

In [77]:
#merged_data[merged_data['customerID'] == 'BBID_20415'].sort_values(by='transactionDate')

In [109]:
merged_data.head(2)

,customerID,Gender,transactionDate,store_code,totalSpend,age,day_of_transaction
0,BBID_2041,1,2016-01-19,2655,672.0,36,1
1,BBID_2041,1,2016-06-21,2655,40.0,36,1


In [78]:
merged_data.nunique()

customerID             25
Gender                  2
transactionDate        58
store_code              4
product_code          376
totalSpend             49
age                    15
day_of_transaction      7
dtype: int64

In [79]:
total = merged_data.isnull().sum().sort_values(ascending=False)
percent = (merged_data.isnull().sum()/merged_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(6)

,Total,Percent
day_of_transaction,0,0.0
age,0,0.0
totalSpend,0,0.0
product_code,0,0.0
store_code,0,0.0
transactionDate,0,0.0


In [93]:
#drop rows with product code as null
merged_data = merged_data[~pd.isnull(merged_data['product_code'])]

In [94]:
customer_product_mapping = merged_data.drop(['Gender','store_code','totalSpend','age','day_of_transaction',
                                             'transactionDate'], axis=1)

In [95]:
merged_data.shape

(500, 8)

In [96]:
#remove the product_code from merged data and shrink the merged data to have unique values
merged_data.drop(['product_code'], axis=1, inplace=True)

In [102]:
#merged_data[merged_data['customerID'] == 'BBID_20415'].sort_values(by='transactionDate')

In [99]:
merged_data.drop_duplicates(inplace=True)

In [128]:
#Shape after removal of duplicates
merged_data.shape

(69, 7)

In [110]:
merged_data.head(2)

,customerID,Gender,transactionDate,store_code,totalSpend,age,day_of_transaction
0,BBID_2041,1,2016-01-19,2655,672.0,36,1
1,BBID_2041,1,2016-06-21,2655,40.0,36,1


In [85]:
customer_product_mapping.shape

(500, 2)

In [86]:
customer_product_mapping.head()

,customerID,product_code
0,BBID_2041,300785147
1,BBID_2041,1000315939
2,BBID_2041,1000314595
3,BBID_2041,300930107
4,BBID_2041,1000010393


In [89]:
customerIds = set(customer_product_mapping['customerID'])

In [90]:
len(customerIds)

25

In [91]:
productCode = set(customer_product_mapping['product_code'])

In [92]:
len(productCode)

376

In [107]:
list(productCode)[0].dtype

dtype('int64')

In [111]:
customer_product_matrix = np.zeros((merged_data.shape[0], len(productCode)))

In [123]:
customer_product_matrix = pd.DataFrame(customer_product_matrix)

In [130]:
#customer_product_matrix

In [137]:
merged_data.shape

(69, 7)